In [1]:
!pip install transformers
!pip install tqdm
!python -m spacy download en_core_web_sm

2023-08-30 06:58:03.373177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 06:58:05.250185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 06:58:08.491783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-30 06:58:08.492433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')
nltk.download('WordNetLemmatizer')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Step 1: Load and preprocess your dataset
# Assuming you have a CSV file with 'text' and 'intent' columns
data = pd.read_csv('/content/output_transformed.csv')
data = data.dropna()
# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['class'], test_size=0.2, random_state=42)

# Step 3: Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Step 4: Train a classification model (Multinomial Naive Bayes)
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

# Step 5: Make predictions on the test data
y_pred = clf.predict(X_test_tfidf)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print a classification report with precision, recall, and F1-score
# print(classification_report(y_test, y_pred))

def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoin the tokens into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


Accuracy: 0.49029126213592233


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index
[nltk_data] Error loading WordNetLemmatizer: Package
[nltk_data]     'WordNetLemmatizer' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Preprocess the new text
new_text = "Can you explain me in one sentence what you are doing?"
new_text_preprocessed = preprocess_text(new_text)  # Apply the same preprocessing steps

# Feature extraction using the same TF-IDF vectorizer
new_text_tfidf = tfidf_vectorizer.transform([new_text_preprocessed])

# Predict the intent label
predicted_intent = clf.predict(new_text_tfidf)[0]

print(f"Predicted Intent: {predicted_intent}")

Predicted Intent: out_of_scope


In [2]:
import spacy
import en_core_web_sm
from sklearn.preprocessing import MinMaxScaler #fixed import
nlp = en_core_web_sm.load()
# Load spaCy with pre-trained GloVe embeddings
# nlp = spacy.load('en_core_web_md')

data = pd.read_csv('/content/output_transformed.csv')
data = data.dropna()

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['class'], test_size=0.2, random_state=42)

def text_to_vector(text):
    doc = nlp(text)
    vector = doc.vector
    return vector

# Vectorize your text data
X_train_vectors = [text_to_vector(text) for text in X_train]
X_test_vectors = [text_to_vector(text) for text in X_test]

scaler = MinMaxScaler()
X_train_vectors = scaler.fit_transform(X_train_vectors)
X_test_vectors = scaler.transform(X_test_vectors)

# Step 4: Train a classification model (Multinomial Naive Bayes)
clf = MultinomialNB()
clf.fit(X_train_vectors, y_train)

# Step 5: Make predictions on the test data
y_pred = clf.predict(X_test_vectors)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print a classification report with precision, recall, and F1-score
# print(classification_report(y_test, y_pred))



def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoin the tokens into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

NameError: ignored

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_vectors = scaler.fit_transform(X_train_vectors)
X_test_vectors = scaler.transform(X_test_vectors)
from sklearn.svm import SVC

svm_clf = SVC(kernel='linear')  # You can try different kernels
svm_clf.fit(X_train_vectors, y_train)

# Predict and evaluate as before
y_pred_svm = svm_clf.predict(X_test_vectors)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'SVM Accuracy: {accuracy_svm}')


SVM Accuracy: 0.6019417475728155


In [7]:
scaler = StandardScaler()
X_train_vectors = scaler.fit_transform(X_train_vectors)
X_test_vectors = scaler.transform(X_test_vectors)
from sklearn.svm import SVC
svm_clf = SVC(kernel='rbf', gamma='scale')  # You can try different kernels
svm_clf.fit(X_train_vectors, y_train)

# Predict and evaluate as before
y_pred_svm = svm_clf.predict(X_test_vectors)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'SVM Accuracy: {accuracy_svm}')

SVM Accuracy: 0.5631067961165048


In [8]:
scaler = StandardScaler()
X_train_vectors = scaler.fit_transform(X_train_vectors)
X_test_vectors = scaler.transform(X_test_vectors)
from sklearn.svm import SVC
svm_clf = SVC(kernel='poly', degree=1)  # You can try different kernels
svm_clf.fit(X_train_vectors, y_train)

# Predict and evaluate as before
y_pred_svm = svm_clf.predict(X_test_vectors)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'SVM Accuracy: {accuracy_svm}')

SVM Accuracy: 0.5


In [13]:
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# Tokenize and obtain BERT embeddings
def get_bert_embeddings(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Batch size of 1
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average the embeddings
    return embeddings

# Vectorize your text data using BERT embeddings
X_train_bert = [get_bert_embeddings(text) for text in X_train]
X_test_bert = [get_bert_embeddings(text) for text in X_test]


In [11]:
!pip3 install "scikit_learn==0.22.2.post1"

In [14]:
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Train a logistic regression classifier
scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)

LR_clf = LogisticRegression(max_iter=1000)
LR_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = LR_clf.predict(X_test_bert)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.7669902912621359


In [15]:
## decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
# Create and train a Decision Tree model
scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)
DT_clf = DecisionTreeClassifier()
DT_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = DT_clf.predict(X_test_bert)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.38349514563106796


In [16]:
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)
RF_clf = RandomForestClassifier()
RF_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = RF_clf.predict(X_test_bert)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.587378640776699


In [17]:
from sklearn.svm import SVC

scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)

SVC_clf = SVC(kernel='rbf', gamma='scale')
SVC_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = SVC_clf.predict(X_test_bert)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.6941747572815534


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = knn_clf.predict(X_test_bert)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.6262135922330098


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:187: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and b

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler #fixed import
scaler = MinMaxScaler()
X_train_bert = scaler.fit_transform(X_train_bert)
X_test_bert = scaler.transform(X_test_bert)
# scaler = StandardScaler()
# X_train_bert = scaler.fit_transform(X_train_bert)
# X_test_bert = scaler.transform(X_test_bert)

mnb_clf = MultinomialNB()
mnb_clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = mnb_clf.predict(X_test_bert)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.6844660194174758


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
# Create a VotingClassifier with your base models
ensemble_model = VotingClassifier(estimators=[
    ('model1', svm_clf),
    ('model2', mnb_clf),
    ('model3', knn_clf),
    ('model4', SVC_clf),
    ('model5', RF_clf),
    ('model6', DT_clf),
    ('model7', LR_clf),

    # Add more base models as needed
], voting='hard')  # 'soft' for weighted voting based on confidence scores

# Fit the ensemble model on the training data
ensemble_model.fit(X_train_bert, y_train)
# Make predictions with the ensemble model
ensemble_predictions = ensemble_model.predict(X_test_bert)

# Fit the ensemble model on the training data
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
ensemble_report = classification_report(y_test, ensemble_predictions)

print(f"Ensemble Model Accuracy: {ensemble_accuracy}")
print("Ensemble Model Classification Report:")
print(ensemble_report)



/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:187: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Ensemble Model Accuracy: 0.7427184466019418
Ensemble Model Classification Report:
                   precision    recall  f1-score   support

        Hand_over       0.00      0.00      0.00         2
  about_onpassive       0.67      0.67      0.67         3
  ask_botcreation       1.00      1.00      1.00         1
          ask_fyn       0.00      0.00      0.00         1
       ask_howold       1.00      1.00      1.00         3
        ask_howru       0.60      0.86      0.71         7
        ask_isbot       0.67      0.29      0.40         7
      ask_ishuman       0.33      0.25      0.29         4
 ask_languagesbot       1.00      1.00      1.00        11
ask_whatspossible       0.67      0.50      0.57         4
        assist_me       0.00      0.00      0.00         4
  contact_details       0.88      1.00      0.94        15
 customer_support       1.00      0.33      0.50         3
          founder       0.83      0.62      0.71         8
     good_evening       1.00    

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_new_data(new_data):
    # Initialize stopwords
    stop_words = set(stopwords.words('english'))

    preprocessed_data = []

    for text in new_data:
        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove punctuation and convert to lowercase
        tokens = [word.lower() for word in tokens if word.isalnum()]

        # Remove stopwords
        tokens = [word for word in tokens if word not in stop_words]

        # Join tokens back into a string
        preprocessed_text = ' '.join(tokens)

        preprocessed_data.append(preprocessed_text)

    return preprocessed_data

# Example usage:
new_data = ["how to delete the profile"]
preprocessed_data = preprocess_new_data(new_data)
print(preprocessed_data)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['delete profile']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [49]:
X_train

1371                        The Try it out is not working
267                      Is it a multi lingual supportive
1063                                Is Rasa really smart?
587                                           I am hungry
1629                      I want to speak binary with you
                              ...                        
347     i have an issue, navigate me to the compliant ...
214                                              hello gm
584                      tell me whereabouts of onpassive
121                       I need help to solve my queries
7251                                         what's 5 + 5
Name: text, Length: 821, dtype: object

In [4]:
# Assuming you've already trained and defined your ensemble model
from sklearn.feature_extraction.text import TfidfVectorizer
import re
# Create the TF-IDF vectorizer with the same parameters as used in training
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Adjust this based on your training data
    stop_words='english',  # Or use the same stop words as during training
    ngram_range=(1, 2)  # Use the same ngram range as during training
)

def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoin the tokens into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Fit the TF-IDF vectorizer on your training data (only once, not on new data)
# X_train_bert_processed = preprocess_text(X_train)  # Replace with your preprocessing function
# tfidf_vectorizer.fit(X_train_bert_processed)

# Preprocess the new text data (replace this with your actual preprocessing steps)
new_text_data = "are you ok"
new_text_data_processed = preprocess_text(new_text_data)  # Replace with your preprocessing function
print(new_text_data_processed,"prinitn")
# Transform the preprocessed text data using the fitted TF-IDF vectorizer
# new_text_data_vectorized = tfidf_vectorizer.transform(new_text_data_processed)

# Make predictions using the ensemble model
ensemble_predictions = ensemble_model.predict(new_text_data_vectorized)

# Print the predictions
print(ensemble_predictions)



NameError: ignored

In [ ]:
# import xgboost as xgb
# scaler = StandardScaler()
# X_train_bert = scaler.fit_transform(X_train_bert)
# X_test_bert = scaler.transform(X_test_bert)
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)

# xgb_clf = xgb.XGBClassifier()
# xgb_clf.fit(X_train_bert, y_train)

# # Make predictions
# y_pred = xgb_clf.predict(X_test_bert)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# print(f"Accuracy: {accuracy}")

In [23]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

clf.fit(X_train_bert, y_train)

# Make predictions
y_pred = clf.predict(X_test_bert)

# Create a simple neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train_bert.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(350, activation='softmax')  # num_classes is the number of classes in your problem
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_bert, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
y_pred = model.predict_classes(X_test_bert)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)
#Please note that you'll need to customize these code templates by loading and preprocessing your specific dataset. Additionally, you may need to adjust hyperparameters and model architectures to suit your problem's requirements.

Epoch 1/10
26/26 - 2s - loss: 3.7954 - accuracy: 0.3886 - 2s/epoch - 86ms/step
Epoch 2/10
26/26 - 0s - loss: 2.6442 - accuracy: 0.4166 - 158ms/epoch - 6ms/step
Epoch 3/10
26/26 - 0s - loss: 2.4824 - accuracy: 0.4166 - 151ms/epoch - 6ms/step
Epoch 4/10
26/26 - 0s - loss: 2.4359 - accuracy: 0.4166 - 194ms/epoch - 7ms/step
Epoch 5/10
26/26 - 0s - loss: 2.3708 - accuracy: 0.4166 - 198ms/epoch - 8ms/step
Epoch 6/10
26/26 - 0s - loss: 2.2885 - accuracy: 0.4397 - 155ms/epoch - 6ms/step
Epoch 7/10
26/26 - 0s - loss: 2.2022 - accuracy: 0.4543 - 154ms/epoch - 6ms/step
Epoch 8/10
26/26 - 0s - loss: 2.1289 - accuracy: 0.4787 - 150ms/epoch - 6ms/step
Epoch 9/10
26/26 - 0s - loss: 2.0015 - accuracy: 0.4921 - 145ms/epoch - 6ms/step
Epoch 10/10
26/26 - 0s - loss: 1.9207 - accuracy: 0.5043 - 172ms/epoch - 7ms/step


AttributeError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assuming you have already made predictions and stored them in y_pred

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Generate a classification report with precision, recall, and F1-score for each class
report = classification_report(y_test, y_pred)
print(report)


In [ ]:
# Preprocess the new text
new_text = "Can you explain me in one sentence what you are doing?"
new_text_preprocessed = preprocess_text(new_text)  # Apply the same preprocessing steps

# Feature extraction using the same TF-IDF vectorizer
new_text_tfidf = tfidf_vectorizer.transform([new_text_preprocessed])

# Predict the intent label
predicted_intent = clf.predict(new_text_tfidf)[0]

print(f"Predicted Intent: {predicted_intent}")